In [ ]:
%matplotlib inline

# Processing a high-resolution Image

This example shows how to use `squidpy.im.process_img`{.interpreted-text
role="func"} to apply any processing function (smoothing, conversion to
grayscale) to a high-resolution image layer of
`squidpy.im.ImageContainer`{.interpreted-text role="class"}.

By default, `squidpy.im.process_img`{.interpreted-text role="func"}
processes the entire input image at once. In the case of high-resolution
tissue slides however, the images might bo too big to fit in memory and
cannot be processed at once. In that case you can use the arguments `xs`
and `ys` that will tile the image in crops of size `(ys, xs)`, process
each crop, and re-assemble the resulting image. Note that you can also
use `squidpy.im.segment_img`{.interpreted-text role="func"} in this
manner.

Note that depending on the processing function used, there might be
border effects occurring at the edges of the crops. In a future version,
we will support the extraction of overlapping crops, which can mitigate
these effects.

For more usage examples see also
`sphx_glr_auto_examples_image_compute_smooth.py`{.interpreted-text
role="ref"},
`sphx_glr_auto_examples_image_compute_gray.py`{.interpreted-text
role="ref"}, and
`sphx_glr_auto_examples_image_compute_segment_fluo.py`{.interpreted-text
role="ref"}.


In [ ]:
import os

import squidpy as sq

import matplotlib.pyplot as plt

# load H&E stained tissue image
img = sq.datasets.visium_hne_image()

First, we crop a smaller image to process


In [ ]:
crop = img.crop_corner(4000, 4000, 2000, 2000)

We will process the image by tiling it in crops of shape
`(ys, xs) = (1000,1000)`.


In [ ]:
sq.im.process_img(crop, img_id="image", processing="gray", xs=1000, ys=1000)

Now we can plot the result


In [ ]:
fig, axes = plt.subplots(1, 2)
axes[0].imshow(crop["image"])
axes[0].set_title("original")
axes[1].imshow(crop["image_gray"], cmap="gray")
axes[1].set_title("converted to grayscale")
for ax in axes:
    ax.axis("off")